# Diffie-Hellman Key Exchange

You have already seen what are multiplicative group of integers (mod N) or the (mod N) group on Day 2

$N = 5$

Number of elements in the group, *aka* its **order** = 4

On one hand,

$$1^1\ mod\ 5 = 1^2\ mod\ 5 = 1^3\ mod\ 5 = ... \equiv 1\ mod\ 5 \Rightarrow Period = 1$$
$$4^1\ mod\ 5 = 4^3\ mod\ 5 = ... \equiv 4\ mod\ 5\ \ \ \ \ \ 4^2\ mod\ 5 = 4^4\ mod\ 5 = ... \equiv 1\ mod\ 5 \Rightarrow Period = 2$$

As long as $x$ and $N$ in $x^1\ (mod\ N), x^2\ (mod\ N), x^3\ (mod\ N),...$ are relatively prime, they will have the periodic property

On the other hand,

$$2^1\ mod\ 5 \equiv 2\ mod\ 5\ \ \ \ \ \ 3^1\ mod\ 5 \equiv 3\ mod\ 5$$
$$2^2\ mod\ 5 \equiv 4\ mod\ 5\ \ \ \ \ \ 3^2\ mod\ 5 \equiv 4\ mod\ 5$$
$$2^3\ mod\ 5 \equiv 3\ mod\ 5\ \ \ \ \ \ 3^3\ mod\ 5 \equiv 2\ mod\ 5$$
$$2^4\ mod\ 5 \equiv 1\ mod\ 5\ \ \ \ \ \ 3^4\ mod\ 5 \equiv 1\ mod\ 5$$

Both have a $Period = 4$. Moreover, *en route* to a value 1, the (mod 5) powers of 2 and 3, each cycle through all the coprimes of 5. Elements of the group whose powers cycle through the whole group are called **primitive roots** (mod N), or alternatively, **generators** of the (mod N) group

2 and 3 are primitive roots (mod 5). And groups that have atleast one generator are called **cyclic**

A note: If the period of $x\ (mod\ N)$ is some number $r$, then $r$ is the smallest number such that $x^r \equiv 1\ (mod\ N)$

*Which (mod N) groups end up being cyclic?*

$$N = 2\ \ \ \ N = 4\ \ \ \ N = power\ of\ an\ odd\ prime\ \ \ \ N = twice\ the\ power\ of\ an\ odd\ prime$$

## Diffie-Hellman Protocol

***Step 1:*** In full view of Eve, Alice and Bob will agree on two things:
1. A modulus $N$
2. A generator $g$ of that group

Say they pick $N = 11$ and $g = 7$ for the sake of illustration

***Step 2:*** Alice and Bob each pick one number, at random, from the group that they won't reveal, even to each other. 

Let's say Alice picks $a = 2$ and Bob picks $b = 4$

***Step 3:*** Alice and Bob compute $g^a\ (mod\ N)$ and $g^b\ (mod\ N)$ respectively and transmit to each other

$$g^a\ (mod\ N) = 7^2\ (mod\ 11) = 5 = A$$
$$g^b\ (mod\ N) = 7^4\ (mod\ 11) = 3 = B$$

***Step 4:*** Alice and Bob each raise the others publicly transmitted ($A = 5$ and $B = 3$) to the power of their own private number (mod N)

$$B^a\ (mod\ N) = 3^2\ (mod\ 11) \equiv 9\ (mod\ 11) \equiv 5^4\ (mod\ 11) = A^b\ (mod\ N)$$

In [21]:
N = 11    # Modulus
g = 7     # Generator

a = 2     # Alice's private key (from the multiplicative group (mod 11))
b = 4     # Bob's private key (from the same multiplicative group (mod 11))

In [22]:
A = ((g**a) % N)    # Alice's transmitted value
B = ((g**b) % N)    # Bob's transmitted value

# To obtain the shared synthesized key
k1 = ((A**b) % N)
k2 = ((B**a) % N)

if (k1 == k2):
    print("The shared synthesized key is " + str(k1))
else:
    print("Check if the generator is valid")

The shared synthesized key is 9


**Why does it work?**

$$A^b\ (mod\ N) = (g^a)^b\ (mod\ N) = (g^b)^a\ (mod\ N) = B^a\ (mod\ N)$$

Given $g$, $a$ and $N$, it is easy to compute $A = g^a\ (mod\ N)$ but given the result $A$, it is hard undo

More generally, when the *modulus* and the *exponents* are enormous, computers can still do the *modular exponentiation* extremely quickly. But there simply is no fast way to *undo modular exponentiation*. That inverse problem is called the **Discrete Logarithm Problem**

# The RSA Encryption Algorithm

Pick 2 random primes $p$ and $q$

Let $p = 5$ and $q = 11$

Calculate $N$ as the product of the two primes $N = p \cdot q = 55$

From the first day of the workshop, *Euler Totient Function* of N: $\phi(N) = (p-1)(q-1) = 40$

Pick another random integer $e$ such that $e < \phi(N)$, and $e$ and $\phi(N)$ are coprime to each other. 

Generate a number $d$ from $p$, $q$ and $e$ such that $e \cdot d \equiv 1\ (mod\ \phi(N))$ and $d < \phi(N)$ through the Extended Euclidean Algorithm from Day 1. There is a unique such $d < \phi(N)$ 

You could use the following relation $d = \frac{k\phi(N) + 1}{e}$ where $d$ is obtained for the smallest $k$ where the numerator is divisible by $e$

Let $e = 7$ and $d = 23$

Here, $e$ is called the *encryption key* and $d$ is called the *decryption key*. Set $(e,N)$ as the public key and keep $d$ as the private key. Remember, $d$ is hard to calculate even though $(e,N)$ are public information

When someone tries to send us a message $M$, they perform the following operation: $M^e\ (mod\ N) = c$ and send the resultant value through the open communication channel.

To decrypt this message, just perform: $c^d\ (mod N) = M$ and obtain $M$, the intended message

In [56]:
import math

def CheckIfPrime(a,b):    # To check if the given p and q are primes
    n = int(math.sqrt(a))
    m = int(math.sqrt(b))
    for i in range(2,n):
        if (a % i == 0):
            print("Try again by using a prime number for p")
            return
    for i in range(2,m):
        if (b % i == 0):
            print("Try again by using a prime number for q")
            return
    print("Given p and q are prime")
    return

def CheckKeyInverse(e,d,phiN):
    if ((e*d) % phiN == 1):
        print("Given (e,d) pair is valid")
    else:
        print("Try again by using a valid (e,d) pair")
    return

def Encrypt(e,N,p):
    c = ((p**e) % N)
    return c

p = 5    # 'Large' Prime 1
q = 11   # 'Large' Prime 2
CheckIfPrime(p,q)
N = p*q  # Product of Primes
phiN = (p-1)*(q-1)

e = 7    # Encryption key
d = 23   # Decryption key
CheckKeyInverse(e,d,phiN)

print()    # Print a blank line

# For the sake of demonstration, try using only small values of M here (M<N). You can explore further in the Assignment Notebooks
M = 2    # Message intended to communicate

plaintext = M
print("Plaintext: " + str(plaintext))
ciphertext = Encrypt(e,N,plaintext)
print("Ciphertext: " + str(ciphertext))

Given p and q are prime
Given (e,d) pair is valid

Plaintext: 2
Ciphertext: 18


In [57]:
def Decrypt(d, N, c):
    p = ((c**d) % N)
    return p

decryptedtext = Decrypt(d,N,ciphertext)
print("Decryptedtext: " + str(decryptedtext))

Decryptedtext: 2


**Why does it work?**

Message $M$

Public Key $(e,N)$

To encrypt it

Ciphertext $c = M^e\ (mod\ N)$

They send this ciphertext via an open channel to you

To decrypt it

plaintext $p = c^d\ (mod\ N) = (M^e)^d\ (mod\ N) = M^{ed}\ (mod\ N)$

Given $e\cdot d \equiv 1\ (mod\ N) \Rightarrow ed - 1 = k\cdot\phi(N) \Rightarrow ed = k\phi(N) + 1$

$\Rightarrow M^{k\phi(N) + 1} = M^{k\phi(N)}\cdot M^1$

$\Rightarrow p = (M^{\phi(N)})^k \cdot M^1\ (mod\ N)$

From *Euler's Totient Theorem*, $a^{\phi(N)} \equiv 1\ (mod\ N)$ when $a$ and $N$ are coprime

$\Rightarrow p = 1 \cdot M^1\ (mod\ N) = M$

# Shor's Algorithm

Let's take the number $N$ which is a product of two primes $p$ and $q$ but now assume that you don't know anything about those primes and your job is to find them. Then here's how you will do it:

***Step 1:*** Pick any number smaller than $N$. Let's call the number you selected $a$. Check to make sure that $a$ and $N$ are relatively prime through the Euclidean Algorithm. 

***Step 2:*** Compute the period, $r$, of $a\ (mod\ N)$.

For the sake of an example, let's say you're trying to find the factors of 35. So $N = 35$ and you pick $a = 8$ since it is relatively prime to 35. Then, with a little computation, we can see that $r = 4$.

Check if $r$ is even and $(a^{r/2} + 1) \ncong 0\ (mod\ N)$. If either of these things fail, we need to go back to *Step 1* and pick a new value of $a$.

Probablistically, there's at least a 50% chance, you'll pick a good value of $a$. So, on average, you won't have to try too many times.

***Step 3:*** Some algebra
$$a^r \equiv 1\ (mod\ N)$$
$$a^r - 1 \equiv 0\ (mod\ N)$$
Saying that something is 0 (mod N) is the same as saying that it's a multiple of $N$. So there must exist some integer $k$ such that 
$$a^r - 1 = k\cdot N$$
Since we assumed $r$ is an even number
$$(a^{r/2} - 1)(a^{r/2} + 1) = kN = kpq$$

For the example, where we are trying to find the factors of 35. Since the period of 8 (mod 35) is 4, we have $$8^4 \equiv 1\ (mod\ 35)$$
So $$8^4 - 1 = 4096 - 1 = 4095 \equiv 0\ (mod\ 35)$$
$$\Rightarrow 8^4 - 1 = k\cdot 35$$
for some $k$. Again, we could solve for $k$ in this case, but it's irrelevant, so I'll leave it as a variable. 
Rewrite this as
$$(8^2 - 1)(8^2 + 1) = kpq$$
where $p$ and $q$ are the prime factors of 35, that we are searching for

***Step 4:*** The greatest common divisor of $a^{r/2} - 1$ and $N$ is one of the prime factors
$$p = gcd(a^{r/2} - 1, N)$$
and similarly
$$q = gcd(a^{r/2} + 1, N)$$

**Why?** The equation $(a^{r/2} - 1)(a^{r/2} + 1) = kpq$ means that $p$ must divide one of the factors on the left and $q$ must divide one of the factors on the left. But, they can not divide the same factor, since that factor would be divisible by $N$. 

**Why is neither factor divisible by N?** For one, we assumed $a^{r/2} + 1 \ncong 0\ (mod\ N)$. For the other, we know $r$ is the minimum value of $x$ such that $a^x \equiv 1\ (mod\ N)$. So $a^{r/2} - 1 \ncong 0\ (mod\ N)$. 

Since $p$ and $q$ divide separate factors on the left side of the equation, we can assume $p$ divides $(a^\frac{r}{2} - 1)$ and $q$ divides $(a^\frac{r}{2} + 1)$. So, our formulas work. 
So, in our example, $p = gcd(63,35) = 7$ and $q = gcd(65,35) = 5$, and they are correct. 